In [36]:
es_host = 'localhost:9200'
import elasticsearch, elasticsearch.helpers
es = elasticsearch.Elasticsearch([es_host])

import subprocess
import socket
import traceback
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# server_address = ('localhost', 40000)

def cmd_out(cmd):
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()

    ## Wait for date to terminate. Get return returncode ##
    p_status = p.wait()
    return output.decode('utf-8').strip()

def get_worker_id():
    global sock
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    mac = cmd_out("ip link show dev `ip route get 1.2.3.4 | grep 1.2.3.4 | awk '{print $5}'` | grep ether | awk '{print $2}' | tr -d :")
    hostname = cmd_out('hostname')
    init_port = 40000
    while True:
        try:
            sock.bind(('localhost', init_port))
            break
        except:
            traceback.print_exc()
            init_port += 1
    return '%s.%s.%d'%(hostname, mac, init_port)

worker_id = get_worker_id()

def get_own_work():
    body = {"query":{"bool":{"must":[{"term":{"worker":{"value":worker_id}}},{"term":{"status":{"value":"downloading"}}}]}}}
    al = es.search(index='map_work', size=1, body=body)
    if not al['hits']['hits']:
        return None
    row = al['hits']['hits'][0]
    return row['_id']

def get_global_work():
    body = {"query":{"bool":{"must":[{"term":{"worker":{"value":"none"}}},{"term":{"status":{"value":"waiting"}}}]}}}
    while True:
        al = es.search(index='map_work', size=1, body=body, version=True)
#         print(al)
        if not al['hits']['hits']:
            return None
        else:
            row = al['hits']['hits'][0]
            ver = row['_version']
            try:
                es.update(index='map_work', doc_type='work', id=row['_id'], version=ver, body={
                        'doc':{'worker':worker_id, 'status':'downloading'}
                    }, refresh=True)
                return row['_id']
            except elasticsearch.ConflictError:
                continue

def push_work(level):
    body = {'levels':level, 'status':'downloading', 'worker':worker_id, 'level':len(level)}
    es.index(index='map_work', doc_type='work', id=level, body=body)

def work_done(level):
    es.update(index='map_work', doc_type='work', id=row['_id'], body={
                        'doc':{'status':'done'}}, refresh=True)

In [40]:
es.index?

In [ ]:
elasticsearch.Con

In [24]:
try:
    es.update(index='abc', doc_type='abc', id='1234', version=14, body={
                        'doc':{'age':39}
                    })
except elasticsearch.ConflictError:
    print("Oops")

Oops


In [34]:
get_global_work()

{'took': 7, 'timed_out': False, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}, 'hits': {'total': 37629, 'max_score': 0.00040342915, 'hits': [{'_index': 'map_work', '_type': 'work', '_id': '021020313020', '_version': 1, '_score': 0.00040342915, '_source': {'levels': '021020313020', 'level': 12, 'status': 'waiting', 'worker': 'none'}}]}}


'021020313020'

In [39]:
get_own_work()

'021020302031'

In [44]:
import time
time.time()

1587264237.5675504

In [45]:
import os.path

In [46]:
os.path.join('a', 'b')

'a/b'

In [47]:
d = 'YURtOEglTWRB'

In [3]:
import base64

In [53]:
base64.decodestring(d.encode('utf-8'))

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: decodestring() is a deprecated alias since Python 3.1, use decodebytes()
  """Entry point for launching an IPython kernel.


b'aDm8H%MdA'

In [2]:
import requests

In [59]:
resp = requests.request('POST', 'http://192.168.1.1/asp/GetRandCount.asp')

In [60]:
resp.content

b'\xef\xbb\xbf95d8c7da9fe6e8b95abd2646906cc64a'

In [61]:
len(resp.content)

35

In [67]:
base64.encodebytes(b'abcd')

b'YWJjZA==\n'

In [33]:
import re

p = re.compile('<input type="hidden" name="onttoken" id="hwonttoken" value="(.*)">')

def reset_router():
    s = requests.Session()
    resp = s.request('POST', 'http://192.168.1.1/asp/GetRandCount.asp')
    token = resp.content[-32:].decode('utf-8')
    pw = b'aDm8H%MdA'
    pw = base64.encodebytes(pw).decode('utf-8')
    pw = pw.strip()
    user = 'CMCCAdmin'
    print(pw)
    resp = s.request('POST', 'http://192.168.1.1/login.cgi', data={'UserName':user,'PassWord':pw, 'x.X_HW_Token':token})
    resp = s.request('GET', 'http://192.168.1.1/html/ssmp/devmanage/cmccdevicereset.asp')
#     print(resp.status_code, resp.content.decode('utf-8'), resp.headers)
    page = resp.content.decode('utf-8')
    g = p.search(page)
    print(g.group(1))
    token = g.group(1)
    try:
        resp = s.request('POST', 'http://192.168.1.1/html/ssmp/devmanage/set.cgi?x=InternetGatewayDevice.X_HW_DEBUG.SMP.DM.ResetBoard&RequestFile=html/ssmp/devmanage/cmccdevicereset.asp', data={'x.X_HW_Token':token}, timeout=10)
    except:
        pass ## device rebooted, we alway timed out
#     print(resp.status_code, resp.content.decode('utf-8'), resp.headers)
    
reset_router()

YURtOEglTWRB
d7aa8df065e41795c093a639dac5a865


In [ ]:
cx = b'\xef\xbb\xbf<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n<meta http-equiv="Pragma" content="no-cache" />\r\n<link rel="stylesheet"  href=\'../../../../Cuscss/frame.css?2019071519300762-1425773353\' type=\'text/css\'>\r\n<script language="JavaScript" src="../../../resource/common/util.js?2019071519300762-1425773353"></script>\r\n<script language="JavaScript" type="text/javascript">\r\n\r\nfunction LoadFrame()\r\n{\r\n}\r\n\r\n\r\nfunction Reboot()\r\n{\r\nif(ConfirmEx("\xe7\xa1\xae\xe5\xae\x9a\xe8\xa6\x81\xe9\x87\x8d\xe5\x90\xaf\xe8\xae\xbe\xe5\xa4\x87\xe5\x90\x97?"))\r\n{\r\nsetDisable(\'Restart_button\',   1);\r\nsetDisable(\'Restore_button\',    1);\r\nsetDisable(\'btnRestoreDftCfg\', 1);\r\nvar Form = new webSubmitForm();\r\n\r\nForm.setAction(\'set.cgi?x=\' + \'InternetGatewayDevice.X_HW_DEBUG.SMP.DM.ResetBoard\'\r\n + \'&RequestFile=html/ssmp/devmanage/cmccdevicereset.asp\');\r\nForm.addParameter(\'x.X_HW_Token\', getValue(\'onttoken\'));\r\nForm.submit();\r\n}\r\n}\r\n\r\n</script>\r\n</head>\r\n<body class="mainbody" onLoad="LoadFrame();">\r\n\r\n<div class="func_spread"></div>\r\n<div class="title_with_desc">\r\n<label id="Title_reboot_lable">\xe8\xae\xbe\xe5\xa4\x87\xe9\x87\x8d\xe5\x90\xaf</label>\r\n</div>\r\n<div class="title_01" style="padding-left:10px;" width="100%">\r\n<label id="Title_reboot_tips_lable">\xe7\x82\xb9\xe5\x87\xbb\xe5\xa6\x82\xe4\xb8\x8b\xe6\x8c\x89\xe9\x92\xae\xe9\x87\x8d\xe5\x90\xaf\xe8\xb7\xaf\xe7\x94\xb1\xe5\x99\xa8\xe3\x80\x82</label>\r\n</div>\r\n<div class="button_spread"></div>\r\n<div align="right">\r\n<input type="hidden" name="onttoken" id="hwonttoken" value="1943ef6eedff9ac04a71f3b2c8b3eeae">\r\n<input type=\'button\' name="Restart_button" id="Restart_button" class="submit" style="width:98px" onClick=\'Reboot()\' value="\xe9\x87\x8d\xe5\x90\xaf">\r\n</div>\r\n</body>\r\n</html>\r\n'
cx = cx.decode('')

In [31]:
s = requests.Session()
s.params = {'timeout':10}
s.params
# requests.Session?
s.options()

TypeError: options() missing 1 required positional argument: 'url'

In [41]:
elasticsearch

<module 'elasticsearch' from '/usr/local/lib/python3.6/dist-packages/elasticsearch/__init__.py'>

In [40]:
es.update?